<a href="https://colab.research.google.com/github/GabrielButti/Projeto-Chatbot/blob/main/Projeto_Chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install langchain==0.3.0
!pip install langchain.groq==0.2.0
!pip install langchain-community==0.3.0
!pip install youtube-transcript-api==0.6.2
!pip install pypdf==5.0.0
!pip install pandas
!pip install numpy==1.23.5

In [ ]:
import os
from langchain_groq import ChatGroq
from langchain.prompts import ChatPromptTemplate

api_key = ''
os.environ['GROQ_API_KEY'] = api_key

chat = ChatGroq(model='llama-3.3-70b-versatile')

def respostas_bot_chat(mensagens):
  mensagens_modelo = [('system', 'Você é um assistente virtual amigável e utilitário')]
  mensagens_modelo += mensagens
  template = ChatPromptTemplate.from_messages(mensagens_modelo)
  chain = template | chat
  return chain.invoke({}).content

def respostas_bot(mensagens, documento):
  mensagem_sistema = '''Você é um assistente virtual amigável e utilitário.
  Você utiliza as seguintes informações para formular suas respostas: {informacoes}'''
  mensagens_modelo = [('system', mensagem_sistema)]
  mensagens_modelo += mensagens
  template = ChatPromptTemplate.from_messages(mensagens_modelo)
  chain = template | chat
  return chain.invoke({'informacoes': documento}).content

In [ ]:
def iniciar_chat_simples():
    mensagens = []
    while True:
        pergunta = input('Usuário: ')
        if pergunta.lower() == "x":
            break
        mensagens.append(('user', pergunta))
        resposta = respostas_bot_chat(mensagens)
        mensagens.append(('assistant', resposta))
        print(f'Bot: {resposta}')

    print('Muito Obrigado por falar comigo!')

def iniciar_chat_com_atribuicoes(documento):
    mensagens = []
    while True:
        pergunta = input('Usuário: ')
        if pergunta.lower() == "x":
            break
        mensagens.append(('user', pergunta))
        resposta = respostas_bot(mensagens, documento)
        mensagens.append(('assistant', resposta))
        print(f'Bot: {resposta}')

    print('Muito Obrigado por falar comigo!')


In [ ]:
from langchain_community.document_loaders import WebBaseLoader
from langchain.document_loaders import YoutubeLoader
from langchain_community.document_loaders import PyPDFLoader
import pandas as pd
from langchain.schema import Document

def carrega_site():
  url_site = input("Digite a URL do Site: ")
  try:
    loader = WebBaseLoader(url_site)
    lista_documentos = loader.load()
    documento = ''
    documento = ''.join([doc.page_content for doc in lista_documentos])
    return documento
  except Exception as e:
      print(f"Ocorreu um erro ao carregar o Site: {e}")
      return None

def carrega_pdf():
  caminho_pdf = input("Digite o caminho para o Arquivo PDF: ")
  try:
    loader = PyPDFLoader(caminho_pdf)
    lista_documentos = loader.load()
    documento = ''
    documento = ''.join([doc.page_content for doc in lista_documentos])
    return documento
  except Exception as e:
      print(f"Ocorreu um erro ao carregar o Arquivo PDF: {e}")
      return None

def carrega_youtube():
  url_youtube = input("Digite a URL do Vídeo do Youtube: ")
  try:
    loader = YoutubeLoader.from_youtube_url(url_youtube, language=['pt'])
    lista_documentos = loader.load()
    documento = ''
    documento = ''.join([doc.page_content for doc in lista_documentos])
    return documento
  except Exception as e:
      print(f"Ocorreu um erro ao carregar o Vídeo no Youtube: {e}")
      return None

def carrega_planilha():
  caminho_arquivo = input("Digite o caminho da Planilha Excel: ")
  try:
    df = pd.read_excel(caminho_arquivo)
    lista_documentos = [Document(page_content=str(linha)) for linha in df.to_dict(orient="records")]
    documento = ''
    documento = ''.join([doc.page_content for doc in lista_documentos])
    return documento
  except Exception as e:
      print(f"Ocorreu um erro ao carregar a Planilha Excel: {e}")
      return None

In [ ]:
print('Bem-Vindo(a) ao Chatbot.')

texto_selecao = '''Digite 1 caso queira analisar um Site.
Digite 2 caso queira analisar um Arquivo PDF.
Digite 3 caso queira analisar um Vídeo no Youtube.
Digite 4 caso queira analisar uma Planilha Excel.
Digite 5 caso queira conversar com o Chat.
Digite x para finalizar o Chatbot.
'''

while True:
  selecao = input(texto_selecao)
  match selecao:
      case '1':
          documento = carrega_site()
          iniciar_chat_com_atribuicoes(documento)
      case '2':
          documento = carrega_pdf()
          iniciar_chat_com_atribuicoes(documento)
      case '3':
          documento = carrega_youtube()
          iniciar_chat_com_atribuicoes(documento)
      case '4':
          documento = carrega_planilha()
          iniciar_chat_com_atribuicoes(documento)
      case '5':
          iniciar_chat_simples()
      case 'x':
        print('Muito Obrigado por falar comigo!')
        break
      case _:
          print("Opção inválida, tente novamente.")
          continue  # Volta para o início do loop se a opção for inválida

  break  # Sai do loop se uma opção válida for selecionada